## Pasos generales en un proyecto de análisis de datos: 

1. Leer los datos: web scrapping, ficheros, apis, bases de datos, etc. 
2. Explorar los datos:
   1. Tamaño 
   2. Columnas
   3. Valores nulos o duplicados
   4. Valores de las variables 
   5. hacer transformaciones: cambiar tipos de datos, cambiar valores, eliminar columnas
3. Unir información (unir df), limpiezas, filtros
4. Analisis:
   1. Correlaciones entre nuestras variables
   2. Visualizaciones 
   3. Outliers
   4. Estadistica descriptiva
5. Automatizacion del flujo de trabajo (ETL)

In [4]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [5]:
df = pd.read_csv("./files/bank-additional.csv", index_col = 0)
df.head()

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,date,latitude,longitude,id_
0,NaN,housemaid,MARRIED,basic.4y,0.0,0.0,0.0,telephone,261,1,999,0,NONEXISTENT,1.1,"93,994","-36,4","4,857",5191,no,2-agosto-2019,41.495,-71.233,089b39d8-e4d0-461b-87d4-814d71e0e079
1,57.0,services,MARRIED,high.school,NaN,0.0,0.0,telephone,149,1,999,0,NONEXISTENT,1.1,"93,994","-36,4",NaN,5191,no,14-septiembre-2016,34.601,-83.923,e9d37224-cb6f-4942-98d7-46672963d097
2,37.0,services,MARRIED,high.school,0.0,1.0,0.0,telephone,226,1,999,0,NONEXISTENT,1.1,"93,994","-36,4","4,857",5191,no,15-febrero-2019,34.939,-94.847,3f9f49b5-e410-4948-bf6e-f9244f04918b
3,40.0,admin.,MARRIED,basic.6y,0.0,0.0,0.0,telephone,151,1,999,0,NONEXISTENT,1.1,"93,994","-36,4",NaN,5191,no,29-noviembre-2015,49.041,-70.308,9991fafb-4447-451a-8be2-b0df6098d13e
4,56.0,services,MARRIED,high.school,0.0,0.0,1.0,telephone,307,1,999,0,NONEXISTENT,1.1,"93,994","-36,4",NaN,5191,no,29-enero-2017,38.033,-104.463,eca60b76-70b6-4077-80ba-bc52e8ebb0eb


### Entender los datos

In [22]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'duration', 'campaign', 'pdays', 'previous', 'poutcome',
       'emp.var.rate', 'euribor3m', 'nr.employed', 'y', 'date', 'latitude',
       'longitude', 'id_'],
      dtype='object')

In [10]:
df.describe().T ## muestra info para variables numericas

,count,mean,std,min,25%,50%,75%,max
age,37880.0,39.977112,10.437957,17.000,32.00000,38.0000,47.00000,98.000
default,34019.0,0.000088,0.009390,0.000,0.00000,0.0000,0.00000,1.000
housing,41974.0,0.535998,0.498708,0.000,0.00000,1.0000,1.00000,1.000
loan,41974.0,0.155620,0.362499,0.000,0.00000,0.0000,0.00000,1.000
duration,43000.0,257.739279,258.666033,0.000,102.00000,179.0000,319.00000,4918.000
campaign,43000.0,2.567233,2.772294,1.000,1.00000,2.0000,3.00000,56.000
pdays,43000.0,962.330953,187.260394,0.000,999.00000,999.0000,999.00000,999.000
previous,43000.0,0.174023,0.497366,0.000,0.00000,0.0000,0.00000,7.000
emp.var.rate,43000.0,0.077128,1.573898,-3.400,-1.80000,1.1000,1.40000,1.400
latitude,43000.0,36.856697,7.225948,24.396,30.61475,36.7610,43.11325,49.384


In [9]:
df.describe(include = 'object').T ## para variables categoricas

,count,unique,top,freq
job,42655,11,admin.,10873
marital,42915,3,MARRIED,25999
education,41193,7,university.degree,12722
contact,43000,2,cellular,27396
poutcome,43000,3,NONEXISTENT,37103
cons.price.idx,42529,26,"93,994",7938
cons.conf.idx,43000,26,"-36,4",8020
euribor3m,33744,309,"4,857",2287
nr.employed,43000,11,"5228,1",16980
y,43000,2,no,38156


In [11]:
df['marital'].unique()

array(['MARRIED', 'SINGLE', 'DIVORCED', nan], dtype=object)

In [12]:
df['marital'].value_counts()

MARRIED     25999
SINGLE      12105
DIVORCED     4811
Name: marital, dtype: int64

In [13]:
df['marital'].value_counts(dropna=False, normalize=True)*100

MARRIED     60.462791
SINGLE      28.151163
DIVORCED    11.188372
NaN          0.197674
Name: marital, dtype: float64

In [14]:
df.select_dtypes(exclude=['int', 'float'])

,age,default,housing,loan,duration,campaign,pdays,previous,emp.var.rate,latitude,longitude
0,NaN,0.0,0.0,0.0,261,1,999,0,1.1,41.495,-71.233
1,57.0,NaN,0.0,0.0,149,1,999,0,1.1,34.601,-83.923
2,37.0,0.0,1.0,0.0,226,1,999,0,1.1,34.939,-94.847
3,40.0,0.0,0.0,0.0,151,1,999,0,1.1,49.041,-70.308
4,56.0,0.0,0.0,1.0,307,1,999,0,1.1,38.033,-104.463
...,...,...,...,...,...,...,...,...,...,...,...
19154,NaN,0.0,0.0,0.0,618,2,999,0,1.4,38.147,-105.582
26206,34.0,0.0,1.0,1.0,42,7,999,0,-0.1,49.235,-112.201
15046,NaN,0.0,1.0,0.0,391,2,999,0,1.4,40.679,-120.015
15280,NaN,NaN,0.0,0.0,674,3,999,0,1.4,27.772,-117.518


In [16]:
df.dtypes

age               float64
job                object
marital            object
education          object
default           float64
housing           float64
loan              float64
contact            object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx     object
cons.conf.idx      object
euribor3m          object
nr.employed        object
y                  object
date               object
latitude          float64
longitude         float64
id_                object
dtype: object

In [17]:
df.dtypes.value_counts()

object     12
float64     7
int64       4
dtype: int64

In [ ]:
df.select_dtypes(include=['object'])

In [ ]:
df['marital'].unique()
df['marital'].value_counts()

for col in df.select_dtypes(include=['object']): 
    print(df[col].unique())



In [19]:
for col in df.select_dtypes(include=['float']): 
    print(df[col].unique())

[nan 57. 37. 40. 56. 45. 59. 24. 25. 41. 29. 35. 54. 50. 39. 30. 55. 46.
 49. 34. 52. 58. 32. 38. 44. 42. 60. 51. 48. 47. 31. 33. 53. 43. 36. 28.
 27. 26. 22. 20. 23. 21. 61. 19. 18. 70. 66. 76. 67. 73. 88. 95. 68. 75.
 63. 80. 62. 72. 82. 64. 71. 69. 78. 65. 85. 79. 77. 83. 81. 74. 17. 87.
 91. 86. 98. 94. 84. 92. 89.]
[ 0. nan  1.]
[ 0.  1. nan]
[ 0.  1. nan]
[ 1.1  1.4 -0.1 -0.2 -1.8 -2.9 -3.4 -3.  -1.7 -1.1]
[41.495 34.601 34.939 ... 38.147 40.679 27.772]
[ -71.233  -83.923  -94.847 ... -120.015 -117.518 -105.026]


In [20]:
df.shape

(43000, 23)

In [21]:
df.drop(['cons.price.idx', 'cons.conf.idx'], axis=1, inplace= True ) ## se elimina las columnas del df original

### Valores Nulos

In [ ]:
# Tipos de valores nulos
    # NaN, None, 'Null', 99999-00000 -> es importante no solo con isnull, pero tb con unique o value_counts
# Gestión de nulos
    # eliminar, rellenar, imputar de forma avanzada

In [ ]:
df['marital'].isnull()

In [24]:
df['marital'].isnull().sum()

85

In [25]:
df['age'].isnull().sum()

5120

In [27]:
df.isnull().sum()

age             5120
job              345
marital           85
education       1807
default         8981
housing         1026
loan            1026
contact            0
duration           0
campaign           0
pdays              0
previous           0
poutcome           0
emp.var.rate       0
euribor3m       9256
nr.employed        0
y                  0
date             248
latitude           0
longitude          0
id_                0
dtype: int64

In [29]:
df.isna().sum()/df.shape[0]*100

age             11.906977
job              0.802326
marital          0.197674
education        4.202326
default         20.886047
housing          2.386047
loan             2.386047
contact          0.000000
duration         0.000000
campaign         0.000000
pdays            0.000000
previous         0.000000
poutcome         0.000000
emp.var.rate     0.000000
euribor3m       21.525581
nr.employed      0.000000
y                0.000000
date             0.576744
latitude         0.000000
longitude        0.000000
id_              0.000000
dtype: float64

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43000 entries, 0 to 27570
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   age           37880 non-null  float64
 1   job           42655 non-null  object 
 2   marital       42915 non-null  object 
 3   education     41193 non-null  object 
 4   default       34019 non-null  float64
 5   housing       41974 non-null  float64
 6   loan          41974 non-null  float64
 7   contact       43000 non-null  object 
 8   duration      43000 non-null  int64  
 9   campaign      43000 non-null  int64  
 10  pdays         43000 non-null  int64  
 11  previous      43000 non-null  int64  
 12  poutcome      43000 non-null  object 
 13  emp.var.rate  43000 non-null  float64
 14  euribor3m     33744 non-null  object 
 15  nr.employed   43000 non-null  object 
 16  y             43000 non-null  object 
 17  date          42752 non-null  object 
 18  latitude      43000 non-nu

In [ ]:
df.notnull().sum()

### Valores Duplicados

In [33]:
df.duplicated().sum()

0

In [40]:
df_data = pd.read_csv("./files/data.csv", index_col = 0)
df_data.head(7)

,Weather,Temperature,Wind,Humidity
Day,,,,
Mon,Sunny,12.79,13,30
Tue,Sunny,19.67,28,96
Wed,Sunny,17.51,16,20
Thu,Cloudy,14.44,11,22
Fri,Shower,10.51,26,79
Sat,Shower,11.07,27,62
Sun,Sunny,17.50,20,10


In [35]:
df_data.duplicated().sum()

0

In [36]:
df_data['Weather'].duplicated().sum()

4

In [39]:
df_data.duplicated(subset=['Weather'], keep="last")

Day
Mon     True
Tue     True
Wed     True
Thu    False
Fri     True
Sat    False
Sun    False
dtype: bool

In [42]:
df_data_si_duplicados = df_data.drop_duplicates(subset=['Weather'], keep="first")
df_data_si_duplicados

,Weather,Temperature,Wind,Humidity
Day,,,,
Mon,Sunny,12.79,13,30
Thu,Cloudy,14.44,11,22
Fri,Shower,10.51,26,79


In [44]:
df_data_si_duplicados_3 = df_data.drop_duplicates(subset=['Weather'])
df_data_si_duplicados_3


,Weather,Temperature,Wind,Humidity
Day,,,,
Mon,Sunny,12.79,13,30
Thu,Cloudy,14.44,11,22
Fri,Shower,10.51,26,79


In [43]:
df_data_si_duplicados_2 = df_data.drop_duplicates()
df_data_si_duplicados_2


,Weather,Temperature,Wind,Humidity
Day,,,,
Mon,Sunny,12.79,13,30
Tue,Sunny,19.67,28,96
Wed,Sunny,17.51,16,20
Thu,Cloudy,14.44,11,22
Fri,Shower,10.51,26,79
Sat,Shower,11.07,27,62
Sun,Sunny,17.50,20,10


In [45]:
## copiar dataframes 
df_data_backup = df_data # es una copia por referencia
df_data_backup

,Weather,Temperature,Wind,Humidity
Day,,,,
Mon,Sunny,12.79,13,30
Tue,Sunny,19.67,28,96
Wed,Sunny,17.51,16,20
Thu,Cloudy,14.44,11,22
Fri,Shower,10.51,26,79
Sat,Shower,11.07,27,62
Sun,Sunny,17.50,20,10


In [48]:
df_data.drop(['Wind', 'Humidity'], axis= 1, inplace= True)
df_data

,Weather,Temperature
Day,,
Mon,Sunny,12.79
Tue,Sunny,19.67
Wed,Sunny,17.51
Thu,Cloudy,14.44
Fri,Shower,10.51
Sat,Shower,11.07
Sun,Sunny,17.50


In [49]:
df_data_backup

,Weather,Temperature
Day,,
Mon,Sunny,12.79
Tue,Sunny,19.67
Wed,Sunny,17.51
Thu,Cloudy,14.44
Fri,Shower,10.51
Sat,Shower,11.07
Sun,Sunny,17.50


In [ ]:
df_new = df_data.copy()

### Resumen de Funciones

In [ ]:
# Funciones para obtener información sobre los data sets

df.head()  # Muestra las primeras 5 filas del DataFrame (por defecto, pero puedes especificar un número).  
df.tail()  # Muestra las últimas 5 filas del DataFrame (por defecto, pero puedes especificar un número).  
df.sample()  # Devuelve una fila aleatoria del DataFrame (puedes especificar cuántas filas con `n`).  
df.columns  # Devuelve un índice con los nombres de las columnas del DataFrame.  
df.shape # Devuelve una tupla con el número de filas y columnas del DataFrame (filas, columnas).
df.describe()  # Genera estadísticas descriptivas como media, desviación estándar, etc., para las columnas numéricas.  
df['columna'].unique()  # Devuelve un array con los valores únicos en una columna.  
df['columna'].value_counts()  # Devuelve el conteo de cada valor único en una columna.  
df.select_dtypes(include=[tipo])  # Selecciona columnas que coincidan con el tipo de dato especificado (por ejemplo, `include=['number']`).  
df.drop('columna', axis=1)  # Elimina una columna del DataFrame (por nombre); usa `axis=0` para eliminar filas.  
df.isnull().sum()  # Devuelve la cantidad de valores nulos en cada columna.  
df.info()  # Muestra información del DataFrame como el número de filas, columnas y tipos de datos.  
df.duplicated().sum()  # Devuelve el número total de filas duplicadas en el DataFrame.  
df.duplicated(subset='date').sum()  # Cuenta filas duplicadas basándose solo en la columna 'date'.  
